# Импорты

In [2]:
import copy

import numpy as np

# Константы

In [3]:
MAX_ITERS_COUNT = 10000
EPS = 1e-5

# Процесс Синкхорна

In [4]:
def sinkhorn_process(mat: np.ndarray, r: np.ndarray, c: np.ndarray):
    new_mat = copy.deepcopy(mat)
    iters_count = 0

    for _ in range(MAX_ITERS_COUNT):
        iters_count += 1

        row_scales = r / np.sum(new_mat, axis=1)

        for j, scale in enumerate(row_scales):
            new_mat[j, :] *= scale

        col_scales = c / np.sum(new_mat, axis=0)

        for j, scale in enumerate(col_scales):
            new_mat[:, j] *= scale

        row_sums = np.sum(new_mat, axis=1)

        if np.linalg.norm(row_sums - r) < EPS:
            break

    return iters_count, new_mat

def sinkhorn_doubly_stochastic(mat: np.ndarray):
    return sinkhorn_process(
        mat,
        np.ones(shape=mat.shape[0], dtype=float),
        np.ones(shape=mat.shape[1], dtype=float)
    )

# Запуск на случайных положительных матрицах

In [5]:
def gen_random_matrices(n: int, cnt: int, scale: float, shift: float):
    for _ in range(cnt):
        yield np.random.random((n, n)) * scale + shift

count = 1000

for n in range(2, 11):
    iter_sum = 0

    for mat in gen_random_matrices(n, count, 1.5, 0.5):
        iters, _ = sinkhorn_doubly_stochastic(mat)

        assert iters != MAX_ITERS_COUNT

        iter_sum += iters

    print(f'Avg iterations count for {n}x{n} matrices: {np.round(iter_sum / count, 3)}')

Avg iterations count for 2x2 matrices: 3.524
Avg iterations count for 3x3 matrices: 4.067
Avg iterations count for 4x4 matrices: 4.109
Avg iterations count for 5x5 matrices: 4.038
Avg iterations count for 6x6 matrices: 3.981
Avg iterations count for 7x7 matrices: 3.95
Avg iterations count for 8x8 matrices: 3.897
Avg iterations count for 9x9 matrices: 3.869
Avg iterations count for 10x10 matrices: 3.838


# Вспомогательные функции генерации случайных матриц с нулями

In [105]:
def check_zero_row(mat: np.ndarray):
    for i in range(mat.shape[0]):
        if not np.any(mat[i, :]):
            return True

    return False

def gen_random_matrix_with_zeros(n: int, k: int, scale: float, shift: float):
    mat = np.random.random((n, n)) * scale + shift

    indices = np.random.permutation([(i, j) for i in range(n) for j in range(n)])[:k]

    for i, j in indices:
        mat[i, j] = 0

    return mat

def gen_random_matrices_with_zeros(n: int, k: int, cnt: int, scale: float, shift: float):
    for _ in range(cnt):
        mat = gen_random_matrix_with_zeros(n, k, scale, shift)

        while check_zero_row(mat) or check_zero_row(mat.T):
            mat = gen_random_matrix_with_zeros(n, k, scale, shift)

        yield mat

# Запуск на матрицах с нулями

In [115]:
count = 100

for n in range(4, 11):
    line = []

    for k in range(1, 2 * n + 1):
        iter_sum = 0

        for mat in gen_random_matrices_with_zeros(n, k, count, 1.5, 0.5):
            iters, _ = sinkhorn_doubly_stochastic(mat)

            iter_sum += iters

        line.append(np.round(iter_sum / count, 3))

    print(f'Avg iteration count for {n}x{n} matrices: ', line)

Avg iteration count for 4x4 matrices:  [6.43, 8.93, 611.06, 2210.28, 4907.64, 7503.88, 9301.02, 9600.36]
Avg iteration count for 5x5 matrices:  [5.33, 6.3, 7.72, 8.82, 409.8, 811.39, 1511.9, 2212.5, 3611.54, 5908.15]
Avg iteration count for 6x6 matrices:  [4.7, 5.3, 5.99, 6.5, 7.23, 8.29, 9.13, 109.81, 310.97, 412.11, 612.45, 814.3]
Avg iteration count for 7x7 matrices:  [4.3, 4.85, 5.17, 5.52, 5.78, 6.34, 6.79, 7.27, 7.76, 8.56, 9.08, 9.76, 110.43, 211.12]
Avg iteration count for 8x8 matrices:  [4.12, 4.44, 4.83, 5.03, 5.2, 5.44, 5.67, 5.86, 6.38, 6.38, 6.76, 7.18, 7.7, 7.89, 8.29, 8.74]
Avg iteration count for 9x9 matrices:  [4.0, 4.2, 4.29, 4.59, 4.77, 4.97, 5.25, 5.38, 5.51, 5.67, 6.06, 6.05, 6.42, 6.55, 6.79, 7.12, 7.19, 7.39]
Avg iteration count for 10x10 matrices:  [3.95, 4.09, 4.08, 4.31, 4.44, 4.68, 4.6, 4.84, 5.03, 5.18, 5.32, 5.45, 5.61, 5.58, 5.88, 6.04, 6.19, 6.27, 6.42, 6.54]


# Неформальный вывод

Посмотрим на получившиеся результаты запусков на матрицах различного размера с различным кол-вом нулей (число нулей возрастает слева направо). Если судить "на глазок", среднее число итераций пропорционально плотности нулей в матрице, т.е. величине $\frac{k}{n^2}$, где $k$ - число нулей, а $n$ - размер стороны матрицы. Действительно, при всех размерах матриц число итераций увеличивается слева направо, а при размерах 4x4 и 5x5 число итераций переваливает за тысячу уже при наличии шести нулей.